# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?
Naive Bayes assume that the effect of the value of a predictor (x) on a given class (c) is independent of the values of other predictors. This assumption is called class conditional independence.

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?
One way to obtain a point estimate is to choose the value of x that maximizes the posterior PDF (or PMF). This is called the maximum a posteriori (MAP) estimation. Maximum a posteriori (MAP) estimate of X given Y=y is the value of x that maximizes the posterior PDF。
If the prior probability is the same for all the hypothesis then MAP = ML

#### 3. What is support vector in SVM?
Support vectors are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane. 

#### 4. What is the intuition behind SVM ?
Find the boundary for classification that has the maximum margin

#### 5. Shortly describ what 'random' means in random forest ?
Random in random forest means use a random subset of samples and a random sample of features in each tree.

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?
maximuze the gain 最大化信息增益 <br>
https://arxiv.org/pdf/1603.02754.pdf


### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:
###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  
###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

In [191]:
import pandas as pd
import re
import random

path='/Users/zixiawang/school/kaikeba/作业笔记/开课吧NLP/Project/Project1/Unsupervise_news_abstract/raw_data'
df=pd.read_csv(path+'/sqlResult_1558435.csv',encoding='GB18030')
print (len(df))

89611


In [192]:
source=df.source.value_counts(2).to_frame().reset_index()
source.head()

,index,source
0,新华社,0.877825
1,微博,0.027642
2,中国证券报?中证网,0.006004
3,中国新闻网,0.005859
4,参考消息网,0.004296


In [193]:
df['y']=df.apply(lambda x:  1 if len(re.findall('新华社',str(x.source)))>0  else 0, axis=1)
#lambda x : True if (x > 10 and x < 20) else False
df['y'].value_counts(2)

1    0.87795
0    0.12205
Name: y, dtype: float64

In [194]:
null_content=df[df['content'].isnull()]
null_content.y.value_counts(2)

0    1.0
Name: y, dtype: float64

### all news with content is NAN are not published by 新华社, drop them from the research dataset

In [195]:
print (len(df))
df=df[~df['content'].isnull()]
print (len(df))
df=df[~df['title'].isnull()]
print (len(df))
df['y'].value_counts(2)

89611
87054
87054


1    0.903738
0    0.096262
Name: y, dtype: float64

### 85% for training 15% for testing

In [204]:
df['train']=df.apply(lambda x: 0 if random.uniform(0, 1)<=0.15 else 1, axis=1)
df['train'].value_counts(2)

1    0.853562
0    0.146438
Name: train, dtype: float64

In [205]:
print (df[df['train']==1].y.value_counts(2))
print (df[df['train']==0].y.value_counts(2))

1    0.90387
0    0.09613
Name: y, dtype: float64
1    0.902965
0    0.097035
Name: y, dtype: float64


In [206]:
df['title_content']=df.apply(lambda x: str(x.title) +' '+ str(x.content).replace('\r\n', ''), axis=1)

In [207]:
df['news_length']=df.title_content.apply(lambda x: len(str(x)))
df['news_length'].describe()

count    87054.000000
mean       466.596411
std        664.113557
min          5.000000
25%        156.000000
50%        202.000000
75%        521.000000
max      17529.000000
Name: news_length, dtype: float64

In [210]:
df['title_content'][0]

'小米MIUI 9首批机型曝光：共计15款 此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。当然，关于MIUI 9的确切信息，我们还是等待官方消息。'

In [229]:
df.head()

,id,author,source,content,feature,title,url,y,train,title_content,news_length
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0,1,小米MIUI 9首批机型曝光：共计15款 此外，自本周（6月12日）起，除小米手机6等15款...,192
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,0,1,骁龙835在Windows 10上的性能表现有望改善 骁龙835作为唯一通过Windows ...,346
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm,0,1,一加手机5细节曝光：3300mAh、充半小时用1天 此前的一加3T搭载的是3400mAh电池...,234
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1,1,葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图） 这是6月18日在葡萄牙中部大佩德...,73
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...,0,1,44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随 （原标题：44岁女子跑深圳约会网友被拒，暴...,1621


### Build a gensim model for word embedding using text in  title and content

In [37]:
#df['title_content'].to_csv('../segment/news_text.txt', index=False)

/Users/zixiawang/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [29]:
import re
from collections import Counter
## clean up the segement text before count the frequency

def get_chinese_number_letters(train_file):
    train = train_file
    train = open(train,encoding="utf8", errors='ignore').read().lower()
    #[\u00C0-\u017F] Unicode range for all Latin characters
    #[\u4e00-\u9fff] Unicode range for all chinese characters
    chinese_only=re.findall(r'([\u4e00-\u9fff]+|[A-Z]+|[0-9]+|[a-z]+| [ ])',train)
    
    #chinese_only=re.findall('[\w\s]',train)
    print ('First 20 characters of the train file is :', ' '.join(chinese_only)[:20])
    return ' '.join(chinese_only)

def get_1_gram_count(text):
    print ('Length of cleaned train file: ', len(text))
    TOKENS = text.split(' ')
    print ('Train file contains ', len(TOKENS), ' of words(tokens).')
    _1_gram_word_counts = Counter(TOKENS)
    return _1_gram_word_counts

In [30]:
import time
start_time = time.time()
train_file='../raw/news_text.txt'
clean_file=get_chinese_number_letters(train_file)
_1_gram_word_counts=get_1_gram_count(clean_file)
print ("My program took", time.time() - start_time, "to run")

First 20 characters of the train file is : 小米 miui 9 首批机型曝光 共计 
Length of cleaned train file:  40461622
Train file contains  6032982  of words(tokens).
My program took 4.5439441204071045 to run


In [51]:
text_file = open("../segment/news_segment.txt", "w")
text_file.write(clean_file)
text_file.close()

In [31]:
from gensim.models import word2vec
import multiprocessing

# 如果目录中有多个文件，可以使用PathLineSentences
sentences = word2vec.PathLineSentences('../segment/')
# 设置模型参数，进行训练
import time
start_time = time.time()
model = word2vec.Word2Vec(sentences, size=300, window=5, min_count=2,workers=multiprocessing.cpu_count())
print ("My program took", time.time() - start_time, "to run")

My program took 34.26250696182251 to run


In [32]:
## use model = word2vec.Word2Vec.load("../models/wiki_news.model") to load the model if you need to re-train the model
# 将Word转换成Vec，然后计算相似度 
from gensim.models import word2vec
import multiprocessing

# 如果目录中有多个文件，可以使用PathLineSentences
sentences = word2vec.PathLineSentences('../segment/')
# 设置模型参数，进行训练
import time
start_time = time.time()
model = word2vec.Word2Vec(sentences, size=100, window=5, min_count=5,workers=multiprocessing.cpu_count())
print ("My program took", time.time() - start_time, "to run")
model.save('../models/news.model')

My program took 17.5863139629364 to run


In [33]:
len(model.wv.vocab.items())

83535

In [37]:
model.wv.most_similar('妈妈')

[('茶园采茶忙', 0.8049883842468262),
 ('中科院沈阳自动化研究所科研团队创新实录', 0.7998490929603577),
 ('花海漫步', 0.798293948173523),
 ('透视百色说脱贫', 0.7979933023452759),
 ('上图为', 0.7973495721817017),
 ('水云龙', 0.7942481637001038),
 ('情书', 0.7933979034423828),
 ('黄大年', 0.7933173179626465),
 ('百条街道花成海', 0.79017174243927),
 ('丝绸之路上的西安港是无', 0.786495566368103)]

In [38]:
model.wv.most_similar('美女')

[('现实中', 0.9711506962776184),
 ('特种部队', 0.9706164598464966),
 ('误导消费者', 0.97043377161026),
 ('申报', 0.9696590304374695),
 ('你怎么看', 0.9691023826599121),
 ('合理控制同业存单等同业融资规模', 0.9685747027397156),
 ('推进学习教育常态化制度化', 0.9679213762283325),
 ('令', 0.9673484563827515),
 ('被告人', 0.967084527015686),
 ('研究中心', 0.9664682149887085)]

In [95]:
list(model.wv.vocab.keys())[:10]

['小米', 'miui', '9', '共计', '15', '6', '月', '12', '日', '起']

In [39]:
print(model.wv.similarity('中国', '美国'))
#save the model

0.45370743


### 可以看出只用news 数据做的model 的词向量效果是不行的，于是采用project1 里面已经train 好的wiki+news 的model来做sentences embedding

In [40]:
import sys
sys.path.append('../src')
import data_io, params, SIF_embedding
weightpara = 1e-5  # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
print('读取用在project 1 中wiki和news语料build的中文模型')
words_chi, We_chi = data_io.getWordmap('../models/wiki_news_model2_vector.txt')
word2weight_chi = data_io.getWordWeight('../models/word_count.txt',  # each line is a word and its frequency,
                                                  weightpara)  # word2weight['str'] is the weight for the word 'str'
print('中文模型读取完毕')     

读取用在project 1 中wiki和news语料build的中文模型
中文模型读取完毕


In [41]:
import sys
from scipy.stats import pearsonr
import re
from scipy.stats.stats import pearsonr
sys.path.append('../src')
import data_io, params, SIF_embedding
words = words_chi
word2weight =word2weight_chi
We = We_chi
rmpc = 1  # number of principal components to remove in SIF weighting scheme
weight4ind = data_io.getWeight(words, word2weight)
print('weight4ind finished ')    

weight4ind finished 


In [165]:
def get_embedding(sentences):
    #sentences=[df.title_content]
    # load sentences
    x, m = data_io.sentences2idx_c(sentences, words)
    print('sentences2idx finished ')
    w = data_io.seq2weight(x, m, weight4ind)  # get word weights
    print('seq2weight finished ')
    # set parameters
    param = params.params()
    param.rmpc = rmpc
    # get SIF embedding
    """
    return 所有需要计算similarity的title，全文，句子的embedding。
    paper 里面用的是TruncatedSVD，project 要求我们用PCA方法decomposite
    """
    print('embedding start ')
    embedding = SIF_embedding.SIF_embedding(We, x, w, param,
                                                method='PCA')  # embedding[i,:] is the embedding for sentence i
    print('embedding finished ')
    print( embedding.shape)
    #df['news_embedding']=embedding
    return embedding

In [211]:
train_sentences=[]
test_sentences=[]
for i in df[df.train==1]['title_content']:
    train_sentences.append(i)
print (len(train_sentences))
for i in df[df.train==0]['title_content']:
    test_sentences.append(i)
print (len(test_sentences))

74306
12748


### Get embedding for train sentences

In [213]:
import time
start_time = time.time()
X_train=get_embedding(train_sentences)
print ("My program took", time.time() - start_time, "to run")

sentences2idx finished 
seq2weight finished 
embedding start 
emb shape: (74306, 100)
PCA used for decompostion PCA(copy=True, iterated_power='auto', n_components=1, random_state=0,
    svd_solver='auto', tol=0.0, whiten=False)
embedding finished 
(74306, 100)
My program took 1184.2459480762482 to run


### Get embedding for test_sentences

In [212]:
import time
start_time = time.time()
X_test=get_embedding(test_sentences)
print ("My program took", time.time() - start_time, "to run")

sentences2idx finished 
seq2weight finished 
embedding start 
emb shape: (12748, 100)
PCA used for decompostion PCA(copy=True, iterated_power='auto', n_components=1, random_state=0,
    svd_solver='auto', tol=0.0, whiten=False)
embedding finished 
(12748, 100)
My program took 200.90599513053894 to run


In [214]:
print (X_train.shape, X_test.shape)

(74306, 100) (12748, 100)


In [215]:
y_train=df[df.train==1]['y']
y_test=df[df.train==0]['y']
print (y_train.shape, y_test.shape)

(74306,) (12748,)


### Use classic Logistic Regression Model with L2正则
先尝试最经典，比较robust的逻辑回归

In [222]:
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()

import numpy as np
from sklearn.metrics import classification_report  #  这个用来打印最终的结果，包括F1-SCORE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
params_c = np.logspace(-5,2,15) # 也可以自行定义一个范围

# 逻辑回归 + L2正则， 利用GrisSearchCV
parameter_grid = {'C': params_c}
model= LogisticRegression(penalty='l2').fit(X_train, y_train)
grid_search = GridSearchCV(model,
                           param_grid=parameter_grid,
                           cv=4, n_jobs=5, scoring='f1')
grid_search.fit(X_train, y_train)
print('Best parameters: {}'.format(grid_search.best_params_))

# 输出最好的参数 
print(grid_search.best_params_)

y_test_predict = grid_search.predict(X_test)
print (classification_report(y_test, y_test_predict,digits=4 ))

print ("My program took", time.time() - start_time, "to run")

Best parameters: {'C': 3.1622776601683795}
{'C': 3.1622776601683795}
              precision    recall  f1-score   support

           0     0.9659    0.9846    0.9752      1237
           1     0.9983    0.9963    0.9973     11511

    accuracy                         0.9951     12748
   macro avg     0.9821    0.9905    0.9862     12748
weighted avg     0.9952    0.9951    0.9952     12748

My program took 26.743412971496582 to run


In [228]:
from sklearn.metrics import confusion_matrix
print (confusion_matrix(y_test_predict,y_test))

[[ 1218    43]
 [   19 11468]]


### Result based on logistic Regression: 
#### Accuracy is 99.52%,   F1 score is already pretty high at 99.73 % 

### Try tpot method
Tpot对大的数据运行非常慢，而且Logistic Regression已经取得了很好的F1 score，提升的空间非常有限了，所以我没有等到全部generation run 完就停止了

In [226]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=10, population_size=10, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
#tpot.export('tpot_news_classification.py')

Generation 1 - Current best internal CV score: 0.9892875867033976
Generation 2 - Current best internal CV score: 0.9913466276754643
Generation 3 - Current best internal CV score: 0.9913466276754643
Generation 4 - Current best internal CV score: 0.9913466276754643
Generation 5 - Current best internal CV score: 0.9920094830043561
Generation 6 - Current best internal CV score: 0.9920094830043561


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(LogisticRegression(input_matrix, C=20.0, dual=False, penalty=l2), learning_rate=0.5, max_depth=4, min_child_weight=13, n_estimators=100, nthread=1, subsample=0.8)

Best pipeline: XGBClassifier(LogisticRegression(input_matrix, C=20.0, dual=False, penalty=l2), learning_rate=0.5, max_depth=4, min_child_weight=13, n_estimators=100, nthread=1, subsample=0.8)
0.9963915908377785


#### Tpot 取得的accuracy 是99.20%

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).